In [77]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

def web_driver(position_x=1600, position_y=300):
    options = webdriver.ChromeOptions()
    # Set Chrome options to block images, CSS, fonts, etc.
    chrome_prefs = {
        # "profile.managed_default_content_settings.images": 2,
        # "profile.managed_default_content_settings.stylesheets": 2,
        # "profile.managed_default_content_settings.fonts": 2,
        # "profile.managed_default_content_settings.plugins": 2,
        # "profile.managed_default_content_settings.popups": 2,
        # "profile.managed_default_content_settings.geolocation": 2,
        # "profile.managed_default_content_settings.notifications": 2,
    }
    options.add_experimental_option("prefs", chrome_prefs)
    # options.add_argument("--headless")  # Run headless if you don't need a visible browser
    # options.add_argument('--start-maximized')  # maximized window
    # Set window size: width=800px, height=600px
    options.add_argument("--window-size=800,600")
    # Set window position: x=100px from left, y=50px from top
    options.add_argument(f"--window-position={position_x},{position_y}")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    # Initialize driver
    # return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return webdriver.Chrome(service=Service(r"C:\Chrome_Driver\chromedriver.exe"), options=options)
   


In [78]:
import time

In [156]:
driver = web_driver() # Open a new browser session
driver.get("https://gemini.google.com/app")
driver.maximize_window() # Maximize the browser window
time.sleep(5) # Wait for the page to load
driver.minimize_window() # Maximize the browser window


In [155]:
driver.quit()

In [112]:
def get_prompt_box():
    return driver.find_element(By.XPATH, '//div[@role="textbox" and @aria-label="Enter a prompt here"]')

def get_send_button():
    return driver.find_element(By.XPATH, '//button[contains(@class, "send-button")]')# Click the button

def send_prompt(msg, delay=0.05):
    # Locate the element using XPath
    try:
        text_box = get_prompt_box()
        # Paste text using send_keys
        text_box.send_keys(msg)
    except Exception as e:
        return f"Error occurred while sending prompt: {e}"
    
    time.sleep(delay)
    err = "Button not found: timeout"
    start_time = time.time()
    while (time.time() - start_time) < 3:  #
        try:
            button = get_send_button()
            label = button.get_attribute("aria-label")
            if "Send" in label:
                button.click()
                return "Prompt sent and button clicked successfully"
            # elif "Stop" in label:
            #     print("This is the Stop button")
        except Exception as e:
            err = e
        time.sleep(0.2)  # Wait before trying again

    return f"Error occurred while clicking button: {err}"


In [81]:
# # Copy the response to clipboard
# # Locate the button using its attributes
# button = driver.find_element(By.XPATH, '//button[@aria-label="Copy" and @data-test-id="copy-button"]')

# # Click the button
# button.click()

In [123]:
def get_last_response(last_response_id=None):  
    err = None
    div_text = None
    try:
        button = get_send_button()
        label = button.get_attribute("aria-label")
        if "Stop" in label:
            return {'response': None, 'error': 'still generating', 'last_id': last_response_id}
    except:
        pass

    try: 
        response_containers = driver.find_elements(
            By.XPATH, 
            "//div[starts-with(@class, 'conversation-container')]"
        )
        new_response_id = response_containers[-1].get_attribute("id")
        if new_response_id == last_response_id:
            return {'response': None, 'error': 'no new response', 'last_id': last_response_id}
        
        div_elements = response_containers[-1].find_elements(
            By.XPATH,
            ".//message-content[starts-with(@class,'model-response-text')]"
        )
        div_text = div_elements[-1].text
    except Exception as e:
        err = e
    return {'response': div_text, 'error': err, 'last_id': new_response_id}

In [228]:
driver.minimize_window() # Maximize the browser window

In [253]:
send_prompt("what is the result of 1094*5678")
time.sleep(1)
get_last_response()

{'response': None, 'error': 'still generating', 'last_id': None}

In [283]:
get_last_response()

{'response': 'The result of 1094×5678 is 6,211,732.',
 'error': None,
 'last_id': '6959724e371f4e04'}

In [250]:
print(get_last_response()['response'])

The result of 1234×5678 is 7,006,652.


In [83]:
prompts = [
    "Please give only the result of the math. no the other text. 123 * 156",
    "Translate this sentence into French: 'I love learning new languages.'",
    "List three creative names for a coffee shop.",
    "Generate a short poem about the ocean in under 40 words.",
    "Explain the concept of photosynthesis in simple terms.",
    "Give me five unique startup ideas related to renewable energy.",
    "Provide a single-word synonym for 'quick'.",
    "Write a tweet (max 280 characters) promoting a new fitness app.",
    "Suggest three healthy dinner recipes with chicken.",
    "Correct the grammar in this sentence: 'She don’t like the movie.'",
    "Describe a futuristic city in exactly two sentences.",
    "Write a short motivational quote about perseverance.",
    "Create a simple riddle with its answer.",
    "Explain the difference between machine learning and deep learning briefly.",
    "Generate a list of five interview questions for a web developer role."
]


In [84]:
math_prompts = [
    "Please give only the result of the math, no extra text: 45 + 67",
    "Only answer with the number: 120 - 48",
    "Calculate and give just the result: 13 × 9",
    "Compute 144 ÷ 12 and show only the answer.",
    "Find the value of 87 + 135 (just the number).",
    "What is 300 - 175? Only output the result.",
    "Multiply 22 by 14 and return only the answer.",
    "Divide 560 by 8 and give only the quotient.",
    "Add 999 + 111 and give the result alone.",
    "Subtract 64 from 200 and output only the answer.",
    "Compute 18 × 7, result only.",
    "Calculate 450 ÷ 15 with no explanation.",
    "Give just the result of 32 + 68.",
    "Show only the value of 500 - 349.",
    "Find 25 × 16 and output just the product.",
    "Give only the result of 84 ÷ 7.",
    "Add 777 + 333 and display only the sum.",
    "Subtract 123 from 456 and give just the difference.",
    "Multiply 19 by 21 and give only the product.",
    "Divide 1024 by 16 and show only the result."
]


In [ ]:
def execute_prompts(prompts):
    for prompt in prompts:
        send_prompt(prompt, delay=0.2)
        time.sleep(4)  # Wait for the response to be generated
        response = "" 
        last_response = ""
        while len(response) > len(last_response) or len(response) == 0:
            time.sleep(0.2)
            last_response = response
            response = get_last_response()
        print(f"Prompt: {prompt}\nResponse: {response}\n{'-'*50}\n")

In [ ]:
execute_prompts(prompts)

Prompt: Please give only the result of the math. no the other text. 123 * 156
Response: 19188
--------------------------------------------------

Prompt: Translate this sentence into French: 'I love learning new languages.'
Response: J'adore apprendre de nouvelles langues.
--------------------------------------------------

Prompt: List three creative names for a coffee shop.
Response: Here are three creative names for a coffee shop:
The Daily Grindstone (A nod to the coffee grinder and the daily hustle)
The Alchemist's Brew (Suggests high-quality, expertly crafted coffee)
Third Place Transit (Evokes the idea of the coffee shop as a brief, relaxing stop between home and work)
--------------------------------------------------

Prompt: Generate a short poem about the ocean in under 40 words.
Response: I can try to find an image like that for you, but can't create it right now. It's possible you're signed out or image creation isn't available in your location.
---------------------------

In [71]:
execute_prompts(math_prompts)

Prompt: Please give only the result of the math, no extra text: 45 + 67
Response: 112
--------------------------------------------------

Prompt: Only answer with the number: 120 - 48
Response: 72
--------------------------------------------------

Prompt: Calculate and give just the result: 13 × 9
Response: 117
--------------------------------------------------

Prompt: Compute 144 ÷ 12 and show only the answer.
Response: 12
--------------------------------------------------

Prompt: Find the value of 87 + 135 (just the number).
Response: 222
--------------------------------------------------

Prompt: What is 300 - 175? Only output the result.
Response: 125
--------------------------------------------------

Prompt: Multiply 22 by 14 and return only the answer.
Response: 308
--------------------------------------------------

Prompt: Divide 560 by 8 and give only the quotient.
Response: 70
--------------------------------------------------

Prompt: Add 999 + 111 and give the result al

In [86]:
hard_math_prompts = [
    "First add 145 + 287, then multiply the result by 3. Give only the final number.",
    "Subtract 84 from 260, then divide that result by 4. Output only the answer.",
    "Multiply 23 by 17, then add 150 to the product. Just give the result.",
    "Add 99 and 101, then multiply the sum by 5. Provide only the final value.",
    "Take 450 ÷ 9, then add 275 to that quotient. Return only the answer.",
    "Multiply 18 by 14, then subtract 72. Give just the result.",
    "Add 375 and 225, then divide the total by 25. Output only the final number.",
    "Subtract 312 from 500, then multiply the difference by 6. Only the result.",
    "Multiply 27 by 15, then add 330. Give only the final answer.",
    "Add 640 and 360, then divide by 8. Output the quotient only.",
    "Subtract 150 from 700, then divide by 11. Just the result.",
    "Multiply 33 by 12, then subtract 77. Provide only the number.",
    "Add 256 and 144, then multiply by 2. Give only the answer.",
    "Take 900 ÷ 15, then add 245. Output only the final value.",
    "Multiply 45 by 19, then add 210. Show only the result.",
    "Add 81 and 99, then divide by 9. Give only the final answer.",
    "Subtract 188 from 444, then multiply by 5. Output only the number.",
    "Multiply 29 by 13, then subtract 52. Just give the result.",
    "Add 720 and 280, then divide the total by 16. Provide only the quotient.",
    "Subtract 63 from 270, then multiply the remainder by 4. Output only the final value."
]


In [115]:
execute_prompts(hard_math_prompts)

Prompt: First add 145 + 287, then multiply the result by 3. Give only the final number.
Response: 1296
--------------------------------------------------

Prompt: Subtract 84 from 260, then divide that result by 4. Output only the answer.
Response: 44
--------------------------------------------------

Prompt: Multiply 23 by 17, then add 150 to the product. Just give the result.
Response: 541
--------------------------------------------------

Prompt: Add 99 and 101, then multiply the sum by 5. Provide only the final value.
Response: 1000
--------------------------------------------------

Prompt: Take 450 ÷ 9, then add 275 to that quotient. Return only the answer.
Response: 325
--------------------------------------------------

Prompt: Multiply 18 by 14, then subtract 72. Give just the result.
Response: 180
--------------------------------------------------

Prompt: Add 375 and 225, then divide the total by 25. Output only the final number.
Response: 24
------------------------------

In [74]:
chat_data = [
    "This is my bussiness description. read and later customer will ask you question. just reply as it is your bussiness and your are selling the product. bussiness description: আমাদের ব্যবসার নাম হলো 'TechHub Dhaka', যা ঢাকায় ভিত্তিক অনলাইন ফেসবুক পেজের মাধ্যমে পরিচালিত হয়। আমরা ইয়ারপডস, পাওয়ার ব্যাংক, ব্লেন্ডার মেশিন এবং অন্যান্য ইলেকট্রনিক্স পণ্য বিক্রি করি। ডেলিভারি সম্পূর্ণ ঢাকায় কুরিয়ার সার্ভিসের মাধ্যমে করা হয়, ডেলিভারির চার্জ সাধারণত ৫০-১০০ টাকা। আমাদের রিফান্ড পলিসি হলো: পণ্য খারাপ বা অপ্রাপ্য হলে ৭ দিনের মধ্যে রিফান্ড পাওয়া যাবে। গ্রাহক সেবা সপ্তাহের সব দিন সকাল ১০টা থেকে রাত ৮টা পর্যন্ত সক্রিয়। আমরা নিশ্চিত করি দ্রুত ডেলিভারি, আসল পণ্য, এবং গ্রাহকের সন্তুষ্টি আমাদের প্রথম অগ্রাধিকার।",
    
    "আপনারা কি ইয়ারপডস এখন স্টকে আছে?",
    "ডেলিভারির চার্জ কত হবে যদি আমি গুলশান থেকে অর্ডার করি?",
    "আমি ব্লেন্ডার মেশিন অর্ডার করতে চাই। কি রঙে পাওয়া যাবে?",
    "পাওয়ার ব্যাংক খারাপ হয়ে গেলে রিফান্ড পাবো কি?",
    "কতো দিন এ পণ্যটি ডেলিভারি হবে?",
    "আমি নতুন কাস্টমার, কিভাবে অর্ডার করতে হবে?",
    "আপনারা কি বিকাশ দিয়ে পেমেন্ট গ্রহণ করেন?",
    "আমি আজ অর্ডার করলে কাল কি ডেলিভারি সম্ভব?",
    "আমি গিফট হিসেবে চাচ্ছি, কি প্যাকেজিং অপশন আছে?",
    "আপনারা কি হোলসেলেও বিক্রি করেন?",
    "পণ্যের ওয়ারেন্টি কতদিনের?",
    "আমি অর্ডার কনফার্ম করেছি কিন্তু ডেলিভারি শিডিউল জানতে চাই।",
    "আপনারা কি শুধু ঢাকায় ডেলিভারি দেন নাকি সারা দেশে?"
]


In [76]:
execute_prompts(chat_data)

Prompt: This is my bussiness description. read and later customer will ask you question. just reply as it is your bussiness and your are selling the product. bussiness description: আমাদের ব্যবসার নাম হলো 'TechHub Dhaka', যা ঢাকায় ভিত্তিক অনলাইন ফেসবুক পেজের মাধ্যমে পরিচালিত হয়। আমরা ইয়ারপডস, পাওয়ার ব্যাংক, ব্লেন্ডার মেশিন এবং অন্যান্য ইলেকট্রনিক্স পণ্য বিক্রি করি। ডেলিভারি সম্পূর্ণ ঢাকায় কুরিয়ার সার্ভিসের মাধ্যমে করা হয়, ডেলিভারির চার্জ সাধারণত ৫০-১০০ টাকা। আমাদের রিফান্ড পলিসি হলো: পণ্য খারাপ বা অপ্রাপ্য হলে ৭ দিনের মধ্যে রিফান্ড পাওয়া যাবে। গ্রাহক সেবা সপ্তাহের সব দিন সকাল ১০টা থেকে রাত ৮টা পর্যন্ত সক্রিয়। আমরা নিশ্চিত করি দ্রুত ডেলিভারি, আসল পণ্য, এবং গ্রাহকের সন্তুষ্টি আমাদের প্রথম অগ্রাধিকার।
Response: নমস্কার! 'TechHub Dhaka'-এ আপনাকে স্বাগতম।
আমরা ঢাকা-ভিত্তিক একটি অনলাইন ফেসবুক পেজ, যেখানে আপনি বিভিন্ন ধরনের ইলেকট্রনিক্স পণ্য খুঁজে পাবেন। আমাদের মূল পণ্যের তালিকায় রয়েছে:
ইয়ারপডস (EarPods)
পাওয়ার ব্যাংক (Power Bank)
ব্লেন্ডার মেশিন (Blender Machine)
এছাড়াও অন্যান্য দ

In [ ]:
# driver.close()  # Close current tab

In [151]:
driver.quit() # Close the entire browser session

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import pyperclip

class LOCAL_API():
    def __init__(self):
        self.history = {'last_prompt_no': -1, 'last_response_id': None}
        self.__init__driver()
        
    def create_web_driver(self, position_x=1920, position_y=0, size_x=1400, size_y=600):
        options = webdriver.ChromeOptions()
        # Set Chrome options to block images, CSS, fonts, etc.
        chrome_prefs = {
            "profile.managed_default_content_settings.images": 2,
            "profile.managed_default_content_settings.stylesheets": 2,
            "profile.managed_default_content_settings.fonts": 2,
            "profile.managed_default_content_settings.plugins": 2,
            "profile.managed_default_content_settings.popups": 2,
            "profile.managed_default_content_settings.geolocation": 2,
            "profile.managed_default_content_settings.notifications": 2,
        }
        options.add_experimental_option("prefs", chrome_prefs)
        # options.add_argument("--headless")  # Run headless if you don't need a visible browser
        # options.add_argument('--start-maximized')  # maximized window
        # Set window size: width=800px, height=600px
        options.add_argument(f"--window-size={size_x},{size_y}")
        # Set window position: x=100px from left, y=50px from top
        options.add_argument(f"--window-position={position_x},{position_y}")
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")

        # Initialize driver
        # return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        return webdriver.Chrome(service=Service(r"C:\Chrome_Driver\chromedriver.exe"), options=options)
    
    def __init__driver(self, delay=0.5):
        self.driver = self.create_web_driver()
        self.creation_time = time.time()
        self.driver.get("https://gemini.google.com/app")
        time.sleep(delay)  # wait for page to load

    def quit_driver(self):
        self.driver.quit()
    
    def restart_driver(self):
        try:
            self.quit_driver()
        except: pass
        self.__init__driver()
    
    def clear_history(self):
        self.history = {'last_prompt_no': -1, 'last_response_id': None}

    def get_prompt_box(self):
        return self.driver.find_element(By.XPATH, '//div[@role="textbox" and @aria-label="Enter a prompt here"]')

    def get_send_button(self):
        return self.driver.find_element(By.XPATH, '//button[contains(@class, "send-button")]')# Click the button

    def send_prompt(self, msg, delay=0.01):
        # Locate the element using XPath
        try:
            text_box = self.get_prompt_box()
            # lines = msg.split('\n')   ## method 1 takes time: slow and lazy
            # for i, line in enumerate(lines):
            #     text_box.send_keys(line)
            #     if i < len(lines) - 1:
            #         text_box.send_keys(Keys.SHIFT, Keys.ENTER)

            # pyperclip.copy(msg)      ## method 2: faster: but disturbs clipboard. risky.
            # text_box.click()
            # text_box.send_keys(Keys.CONTROL, 'v')  # paste entire text
            ## method 3: fastest and efficient: does not disturb clipboard, does not need click
            # self.driver.execute_script("arguments[0].innerText = arguments[1];", text_box, msg)
            self.driver.execute_script("arguments[0].textContent = arguments[1];", text_box, msg.rstrip('\n'))
        except Exception as e:
            return f"Error occurred while sending prompt: {e}"
        # print("Prompt sent to box") # delete later
        # time.sleep(delay)  ## no need to wait here
        err = "Button not found: timeout"
        start_time = time.time()
        while (time.time() - start_time) < 3:  # timeout after 3 seconds
            # print("Trying to find button") # delete later
            try:
                button = self.get_send_button()
                label = button.get_attribute("aria-label")
                if "Send" in label:
                    button.click()
                    self.history['last_prompt_no'] += 1
                    self.history[self.history['last_prompt_no']] = {'prompt': msg, 'response': None}
                    return "Prompt sent and button clicked successfully"
                # elif "Stop" in label:
                #     print("This is the Stop button")
            except Exception as e:
                err = e
            time.sleep(0.1)  # Wait before trying again
        return f"Error occurred while clicking button: {err}"
    
    def get_last_response(self):  
        err, div_text, new_response_id = None, None, False
        try:
            button = self.get_send_button()
            label = button.get_attribute("aria-label")
            if "Stop" in label:
                return {'response': None, 'error': 'still generating', 'id': None}
        except:
            pass

        try: 
            response_containers = self.driver.find_elements(
                By.XPATH, 
                "//div[starts-with(@class, 'conversation-container')]"
            )
            new_response_id = response_containers[-1].get_attribute("id")
            if new_response_id == self.history['last_response_id'] and self.history['last_response_id']:
                return {'response': None, 'error': 'no new response', 'id': None}
            
            div_elements = response_containers[-1].find_elements(
                By.XPATH,
                ".//message-content[starts-with(@class,'model-response-text')]"
            )
            div_text = div_elements[-1].text
        except Exception as e:
            err = e
        response = {'response': div_text, 'error': err, 'id':  new_response_id if new_response_id else None}
        if new_response_id:
            self.history[self.history['last_prompt_no']]['response'] = response
            self.history['last_response_id'] = new_response_id
        return response
    
    def copy_last_response_to_clipboard(self):
        pass
        # steps: same as get_last_response, then find the copy button and click it
        # try:
        #     <button _ngcontent-ng-c4159091283 mat-button tabindex="0" mattooltip="Copy response" aria-label="Copy" data-test-id="copy-button" class="mdc-button mat-mdc-button-base mat-mdc-tooltip-trigger icon-button mat-mdc-button mat-unthemed" mat-ripple-loader-class-name="mat-mdc-button-ripple" jslog="178035;track:generic_click,impression;BardVeMetadataKey:[["r_7711edcb96c84e21","c_a39913e6d4337352",null,"rc_c5681538456025e4",null,null,"en",null,1,null,null,1,0]];mutable:true" aria-describedby="cdk-describedby-message-ng-1-11" cdk-describedby-host="ng-1">flex
        # except Exception as e:
        #     return f"Error occurred while copying to clipboard: {e}"

    def execute_prompt(self, prompt, inertial_delay=0.4, timeout=30):
        send_status = self.send_prompt(prompt)
        response = {'response': None, 'error': None, 'id': None}
        result = {'prompt': prompt, 'response': response}
        # print(send_status)
        if "Error" in send_status:
            result['response']['error'] = send_status
            return result
        
        time.sleep(inertial_delay)
        result['response']['error'] = "timeout after sending prompt"
        start_time = time.time()
        while (time.time() - start_time) < timeout:
            response = self.get_last_response()
            if response['error'] == 'still generating':
                time.sleep(0.2)
                continue
            elif response['error'] == 'no new response':
                time.sleep(0.2)
                continue
            else:
                result['response'] = response
                break
        return result

    def execute_prompts(self, prompts, inertial_delay=0.4, timeout=30, delay_between_prompts=0.4):
        results = []
        for prompt in prompts:
            result = self.execute_prompt(prompt, inertial_delay, timeout)
            results.append(result)
            if result['response']['error'] and 'timeout' in result['response']['error']:
                break
            time.sleep(delay_between_prompts)
        return results


In [3]:
api = LOCAL_API()
# api.execute_prompt("what is the result of 1094*5678?\n response like 'ans = <number>'")

In [4]:
prompt = """
translate the questions to english.
write latex code where necessary (i.e. formula of compound [Cr(NH_3)_6]^{3+}). 
write all text in code editor 
insert subscript "_" where needed. (i.e. H_2O)
To keep a space between value and unit insert "\," between. i.e. mass = 400\,g and velocity = 30\,kmh^{-1} and (3,\,0)

write a,b,c,d options in 1 line when options lengths are small
write a,b,c,d options in 2 lines (2 per line) when option length are big

sample output format/style to follow:
01. Which of the following is diamagnetic? 
a. [Cr(NH_3)_6]^{3+}	b. [Ni(CN)_4]^{2-}  
c. [CuCl_4]^{2-}  	d. [CoF_6]^{2-}    

03. Which of the following is paramagnetic?
a. O_2^{-}  b. NO^{+}  c. N_2  d. None of these  

06. What is the oxidation number of Cr in [Cr(H_2O)_4Cl_2]Br ? 
a. +2  b. +3  c. -2  d. +6  

07. Which of the following ion shows paramagnetic property?
a. K^{+}  b. Mg^{2+}  c. Ni^{2+}  d. Cu^{+}  

08. If the electron configuration of the d orbital of a metal compound is given, which of the compound will be more paramagnetic?  
a. 2, 2, 2, 2, 1  b. 1, 1, 1, 1, 1  
c. 2, 1, 1, 1, 1  d. 2, 2, 1, 1, 1  

Questions to translate:
01. লেখচিত্রে একটা বস্তুর দূরত্বের (x) সাথে নির্ভরশীল এর উপর ক্রিয়াশীল বলকে (U) দেখানো হয়েছে । বলের একক N এবং দূরত্বের একক m । x\ =\ 0 হতে x\ =\ 6\ m দূরত্বের মধ্যে বস্তুর উপর কৃতকাজের মান কত হবে?
 
a. 4.5\ J 	   b. 13.5\ J 	c. 9.0\ J 	   d. 18\ J

02. F₁ ও F₂ বলদ্বয় দ্বারা 25\ Nm^{-1} ও 16\ Nm^{-1} স্প্রিং ধ্রুবক বিশিষ্ট দুটি স্প্রিংকে সম্প্রসারিত করা হল । যদি স্প্রিং এর সম্প্রসারণে কৃত কাজের পরিমাণ সমান হয় তবে  \frac{F_1}{F_2}\  এর মান কত হবে?
a. \frac{4}{5}        b. \frac{5}{4}       c. \frac{16}{25}      d. \frac{25}{16}

03. চিত্রে, কৃতকাজ কত একক? 
a. 100\pi		b. 50\pi
c. 25\pi		d. 12.5\pi

04. h মিটার উচ্চতা হতে একটি বস্তু নিচে পড়ে গেল । কোথায় তার গতিশক্তি স্থিতিশক্তির অর্ধেক হবে?
a. ভূমি থেকে \frac{2h}{3}\ m উপরে
b. উপর থেকে \frac{2h}{3} নিচের দিকে
c. ভূমি থেকে \frac{2h}{2}\ m উপরে
d. উপর থেকে \frac{2h}{2}\ m নিচের দিকে

05. m\ kg ভরের একটি লোহার বল আনুভূমিক পথে চলমান ছিল । হঠাৎ এটি বিস্ফোরিত হয়ে দুটি মান ভরের টুকরোতে পরিণত হয় । পূর্বে বলটির ভরবেগ ছিল 3p । বিস্ফোরণের পর একটি টুকরো 4p ভরবেগে উপরের দিকে চলতে শুরু করে । বিস্ফোরণের পর সিস্টেমটি কত গতিশক্তি লাভ করে?
a. \frac{25p^2}{m}	    b. 16p²m	c. 41p²m	   d. \frac{73p^2}{m}
 
06. 2\ kW ক্ষমতার একটি পাম্প এর দক্ষতা 74% । এটি 20m উপরে প্রতি সেকেন্ডে কতটুকু পানি তুলতে পারবে?
a. 8\ kg 	  b. 7.55\ kg   c. 12.31\ kg   d. 1.02\ kg 

07. একটি বস্তু সমদ্রুতিতে বৃত্তাকার পথে ঘুরলে-
a. এর উপর কোন কাজ হয় না	   b. ত্বরণ থাকে না
c. বল কাজ করে না 	   d. বেগ অপরিবর্তিত থাকে

08. 20\ m উঁচু দালান থেকে টেনিস বল গড়িয়ে মাটিতে পড়ার সময় বেগ 22\ ms^{-1} হয় । অনুভূমিক বেগ কত?
a. 4.7 ms^{-1}  		b. 7.29 ms^{-1}
c. 11.21\ ms^{-1} 		d. 9.59 ms^{-1}

09.                   উল্লম্বতলে 100\ gm ভরের একটি বস্তুকে ঘুরিয়ে A থেকে B অবস্থানে আনলে
 শক্তির পরিবর্তন কত? [g\ =\ 10\ ms^{-2}]
a. 20J 	  b. 60J     c. 40J 	d. 10J

10. একটি কুয়ার ব্যাসার্ধ 2\ m , গভীরতা 10\ m , এর \frac{3}{4} অংশ পানি দ্বারা পূর্ণ থাকলে একে সম্পূর্ণ পানিশূন্য করতে কী পরিমাণ কাজ করতে হবে?
a. 5.77\ \times\ 10^6\ J   	b. 6\ kJ
c. 2.19\ x\ 10^6\ J   		d. 4.17\ x\ 10^6\ J 
"""

In [5]:
api.execute_prompt(prompt, timeout=100)

KeyboardInterrupt: 

In [6]:
api.quit_driver()
api = None

In [9]:
import requests
response = requests.get("http://localhost:8000/api/local/?prompt=" + prompt +"&timeout=200")
print(response.json())

{'result': {'prompt': '\ntranslate the questions to english.\nwrite latex code where necessary (i.e. formula of compound [Cr(NH_3)_6]^{3 }). \nwrite all text in code editor \ninsert subscript "_" where needed. (i.e. H_2O)\nTo keep a space between value and unit insert "\\," between. i.e. mass = 400\\,g and velocity = 30\\,kmh^{-1} and (3,\\,0)\n\nwrite a,b,c,d options in 1 line when options lengths are small\nwrite a,b,c,d options in 2 lines (2 per line) when option length are big\n\nsample output format/style to follow:\n01. Which of the following is diamagnetic? \na. [Cr(NH_3)_6]^{3 }\tb. [Ni(CN)_4]^{2-}  \nc. [CuCl_4]^{2-}  \td. [CoF_6]^{2-}    \n\n03. Which of the following is paramagnetic?\na. O_2^{-}  b. NO^{ }  c. N_2  d. None of these  \n\n06. What is the oxidation number of Cr in [Cr(H_2O)_4Cl_2]Br ? \na.  2  b.  3  c. -2  d.  6  \n\n07. Which of the following ion shows paramagnetic property?\na. K^{ }  b. Mg^{2 }  c. Ni^{2 }  d. Cu^{ }  \n\n08. If the electron configuration o